<a href="https://colab.research.google.com/github/thiagodepaulo/nlp/blob/enap/CRF_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install sklearn_crfsuite
import pandas as pd
import sklearn_crfsuite
from sklearn_crfsuite import metrics
import joblib
import math

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.1 MB/s eta 0:00:00


In [6]:
from google.colab import drive
drive.mount('/content/drive')

# Load the labeled data from Google Drive
with open('/content/drive/My Drive/labeled.csv', 'r') as f:
    df = pd.read_csv(f)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
def extract_features(sentence):
  sentence_features = []
  for j in range(len(sentence)):
    word_feat = {
            'word': sentence[j].lower(),
            'capital_letter': sentence[j][0].isupper(),
            'all_capital': sentence[j].isupper(),
            'isdigit': sentence[j].isdigit(),
            'word_before': sentence[j].lower() if j==0 else sentence[j-1].lower(),
            'word_after:': sentence[j].lower() if j+1>=len(sentence) else sentence[j+1].lower(),
            'BOS': j==0,
            'EOS': j==len(sentence)-1
    }
    sentence_features.append(word_feat)
  return sentence_features

def separate_cols(arq):
    x = []
    y = []
    for i in range(len(arq)):
        x.append(extract_features(arq['text'][i].split()))
        y.append(arq['labels'][i].split())
    return x, y

x, y = separate_cols(df)
train_x = x[:math.floor(0.7*len(df))]
train_y = y[:math.floor(0.7*len(df))]
test_x = x[math.floor(0.7*len(df)):]
test_y = y[math.floor(0.7*len(df)):]

len(df), len(train_x), len(train_y), len(test_x), len(test_y)

(5516, 3861, 3861, 1655, 1655)

In [8]:
model = sklearn_crfsuite.CRF(
    algorithm = 'l2sgd',
    c2=1,
    max_iterations=100,
    all_possible_transitions=True,
    verbose=False
)

model.fit(train_x, train_y)
y_pred = model.predict(test_x)

In [9]:
labels = list(model.classes_)
labels.remove('O')

f1 = metrics.flat_f1_score(test_y, y_pred,
                      average='weighted', labels=labels)
print(f1)

0.981261187383036


In [18]:
from sklearn_crfsuite import metrics
from sklearn_crfsuite.utils import flatten
from sklearn import metrics as skmetrics


y_test_flat = flatten(test_y)
y_pred_flat = flatten(y_pred)

print(skmetrics.classification_report(y_test_flat, y_pred_flat))


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                     precision    recall  f1-score   support

              B-ATO       0.99      0.99      0.99      1654
            B-CARGO       0.98      0.92      0.95      1646
           B-CLASSE       0.99      0.99      0.99       877
B-COD_MATRICULA_ATO       0.99      0.89      0.94      1652
      B-EMPRESA_ATO       0.97      0.93      0.95       807
       B-FUND_LEGAL       0.99      0.99      0.99      1655
         B-NOME_ATO       0.98      0.97      0.98      1655
           B-PADRAO       0.97      1.00      0.99      1499
         B-PROCESSO       0.98      0.95      0.96      1516
           B-QUADRO       0.97      0.99      0.98      1536
              I-ATO       1.00      0.99      0.99      1392
            I-CARGO       0.96      0.96      0.96      5013
           I-CLASSE       0.99      0.98      0.98       882
I-COD_MATRICULA_ATO       0.00      0.00      0.00        93
      I-EMPRESA_ATO       0.97      0.93      0.95      4046
       I-FUND_LEGAL    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
